In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import wandb

# Tensorflow

### WandB connection

In [2]:
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: filippetran (ns-super-team). Use `wandb login --relogin` to force relogin


True

### Loading the data

In [3]:
x_train = pd.read_csv('data/train/data.csv').to_numpy(dtype='float32')
y_train = pd.read_csv('data/train/labels.csv').to_numpy(dtype='float32')

x_val = pd.read_csv('data/val/data.csv').to_numpy(dtype='float32')
y_val = pd.read_csv('data/val/labels.csv').to_numpy(dtype='float32')

print(x_train[0])
print(y_train[0])
print()
print(x_val[0])
print(y_val[0])

[  1.      49.       1.       1.     110.8833]
[0.]

[ 1. 39.  0.  0. 50.]
[0.]


### Building a model

In [4]:
model = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=(5,)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')    
    ])

In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                384       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2,497
Trainable params: 2,497
Non-trainable params: 0
_________________________________________________________________


In [6]:
model.compile(
    optimizer=tf.keras.optimizers.SGD(1e-3),
    loss='binary_crossentropy',
    metrics=['accuracy'])

### Training the model

In [7]:
history=model.fit(
    x_train,
    y_train,
    epochs=500,
    validation_data=(x_val, y_val),
    batch_size=16
    )

Epoch 1/500
39/39 [==============================] - 1s 12ms/step - loss: 0.9521 - accuracy: 0.6952 - val_loss: 0.8936 - val_accuracy: 0.5940
Epoch 2/500
39/39 [==============================] - 0s 3ms/step - loss: 0.7042 - accuracy: 0.6742 - val_loss: 0.7629 - val_accuracy: 0.6165
Epoch 3/500
39/39 [==============================] - 0s 3ms/step - loss: 0.6588 - accuracy: 0.6839 - val_loss: 0.6679 - val_accuracy: 0.6090
Epoch 4/500
39/39 [==============================] - 0s 3ms/step - loss: 0.6318 - accuracy: 0.6774 - val_loss: 0.6231 - val_accuracy: 0.6541
Epoch 5/500
39/39 [==============================] - 0s 4ms/step - loss: 0.7005 - accuracy: 0.6677 - val_loss: 0.6419 - val_accuracy: 0.6466
Epoch 6/500
39/39 [==============================] - 0s 3ms/step - loss: 0.6369 - accuracy: 0.6871 - val_loss: 0.6544 - val_accuracy: 0.6241
Epoch 7/500
39/39 [==============================] - 0s 3ms/step - loss: 0.6463 - accuracy: 0.6726 - val_loss: 0.6360 - val_accuracy: 0.6767
Epoch 8/500


In [8]:
import plotly.express as px
fig = px.line({'loss': history.history['loss'], 'val_loss': history.history['val_loss']})
fig.show()

In [9]:
import plotly.express as px
fig = px.line({'loss': history.history['accuracy'], 'val_loss': history.history['val_accuracy']})
fig.show()